In [1]:
import numpy as np
import pandas as pd
from scipy.special import softmax


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


/home/mauricio/miniconda3/envs/sb-sentiment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

print(torch.__version__)

torch.cuda.is_available()

2.2.1


False

## Let's do the EDA.

In [4]:
df = pd.read_csv("youtube_comments.csv", delimiter=',')
df_comments= df["Comment"]

In [5]:
text  = df_comments[:]
text

0                                      USHER IS A LEGEND 🔥
1                                                    Yeah!
2                                           Let Him Cook!!
3                                    Ok so I ate a raccoon
4        All these star performers came together tonigh...
                               ...                        
38296                         Fireeeee asfff usher Ateeee😭
38297                                                first
38298                                          yuppp usher
38299                                          Legendary 🔥
38300                                              Usher??
Name: Comment, Length: 38301, dtype: object

In [6]:
modified_text = []
for i in range(len(text)): 
    # Check if the element is NaN
    if isinstance(text[i], str):
        # Initialize an empty list to store modified words
        modified_words = []
        # Split the text into words
        for word in text[i].split(' '):
            if ' ' not in word and ('@@' in word or word.startswith('http')):
                continue
            elif '@@' in word:
                word = word.replace('@@', '@user')
            elif word.startswith('http'):
                word = 'http'

            # Keep the original word if it doesn't match any conditions
            modified_words.append(word)
        # Check if the list of modified words is empty
        if modified_words:
            # Join the modified words into a single string and append it to modified_text
            modified_text.append(' '.join(modified_words))


In [7]:
len(modified_text)

38072

## load the model


In [8]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

## split the data


In [10]:
labels = ['Negative', 'Neutral', 'Positive']
# # Split data into training and validation sets (e.g., 80% for training, 20% for validation)
# X_train, X_val, y_train, y_val = train_test_split(modified_text, labels, test_size=0.2, random_state=42)


In [13]:
# Initialize variables to store cumulative sentiment scores
total_scores = np.zeros(len(labels))

# Loop through each text in your dataset
for text in modified_text:
    # Encode the comment
    encoded_comment = tokenizer(text, return_tensors='pt')
    
    # Analyze sentiment using the model
    output = model(**encoded_comment)
    
    # Extract sentiment scores
    scores = output[0][0].detach().numpy()
    
    # Apply softmax to convert scores to probabilities
    scores = softmax(scores)
    
    # Accumulate sentiment scores
    total_scores += scores

# Calculate average sentiment scores
average_scores = total_scores / len(modified_text)



RuntimeError: The expanded size of the tensor (956) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 956].  Tensor sizes: [1, 514]

In [ ]:
print(average_scores)

[212.89004262 336.22087285 422.88908344]


In [ ]:
# Print average sentiment scores
for label, score in zip(labels, average_scores):
    print(f"{label}: {score:.4f}")


Negative: 0.2190
Neutral: 0.3459
Positive: 0.4351
